# Exercise 8: Hypothesis Testing 



## Table of Contents

* Parametric tests
    * one-sample t-test
    * paired t-test
    * two-sample t-test
    * ANOVA
* Testing parametric assumptions
    * Verifying assumptions
    * Modifying data 
* Non-parametric tests
    * Signed-rank tests
    * Bootstrapping and estimation plots

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as sps
import os
import statsmodels.formula.api as smf
import statsmodels.api as sm

# For retina displays only 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
%matplotlib inline

## Parametric tests

### one-sample t-test

In [ ]:
# Generating data 
Y = np.random.normal(loc=3, scale=2, size=100)

df_one_samp = pd.DataFrame(data = {'Y': Y})

In [ ]:
# Excersise: What do you expect for pvalue if popmean = 3? What happens as you move away from 3? 

sps.ttest_1samp(Y, popmean=3)

In [ ]:
# Exersise: find p value by using statsmodels and thinking about the test as a model



### Paired t-test

In [ ]:
# Generating data 
N = 100
X1 = np.random.normal(loc=0, scale=2, size=N)
X2 = X1 + np.random.normal(loc=2, scale=1, size=N)


In [ ]:
sps.ttest_rel(X1, X2)

In [ ]:
# Exercise: Use ttest_1samp amd show that you get the same result if you do one-way t-test of the difference






### Two-sample t-test 

In [ ]:
# Generating data 
N = 100
effect_size = 1 
std_dev = 2
X1 = np.random.normal(loc=0, scale=std_dev, size=N)
X2 = np.random.normal(loc=effect_size, scale=std_dev, size=N)

In [ ]:
sps.ttest_ind(X1, X2)

In [ ]:
# Exersise: find p value by using statsmodels and thinking about the test as a model



### ANOVA

In [ ]:
# Generating data 
N = 100
effect_size_2 = 1
effect_size_3 = -3
std_dev = 2
X1 = np.random.normal(loc=0, scale=std_dev, size=N)
X2 = np.random.normal(loc=effect_size_2, scale=std_dev, size=N)
X3 = np.random.normal(loc=effect_size_3, scale=std_dev, size=N)

In [ ]:
sps.f_oneway(X1, X2, X3)

In [ ]:
# Exersise (tougher): find p value by using statsmodels and thinking about the test as a model



## Testing non-parametic assumptions

In [ ]:
# Exercise: Generate some non-gaussian data, make sure you know how to specifcy the mean of the 
# population and your sample size










In [ ]:
# Exercise: Test it ASSUMING it meets assumptions of parametric tests. Do you get the correct result?
# 1. Play around with sample size, variances, etc. 
# 2. If you are feeling really into it, draw a graph showing what happens as one population gets 
# further and further from normal. 







In [ ]:
# Exercise: Look up one of the following and apply it to your data: 
# 
# Data are normally distributed
#    D’Agostino-Pearson
#    Shapiro-Wilk
#    Kolmogorov-Smirnov
#    Lilliefors Test
# Equal variance between groups
#    Levene’s Test





## Non-parametric tests

### Signed-rank tests

### Bootstrapping and estimation plots